In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os, random

print(tf.__version__)

BATCH_SIZE = 32
IMG_SIZE = (300, 300)

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

2.4.0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15207368766536663269
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1408043827
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6494235849172142852
physical_device_desc: "device: 0, name: GeForce MX250, pci bus id: 0000:3a:00.0, compute capability: 6.1"
]


In [2]:
model = tf.keras.applications.InceptionResNetV2(include_top=False, input_shape=(300, 300, 3), pooling='avg')

In [3]:
import pandas as pd

train_label_MBTI = pd.read_csv("csv\\original\\train_MBTI.csv", usecols=[1], delimiter=',').to_numpy().flatten()
test_label_MBTI = pd.read_csv("csv\\original\\test_MBTI.csv", usecols=[1], delimiter=',').to_numpy().flatten()

inception_test_label = pd.read_csv("csv\\test\\MBTI_label.csv", usecols=[1], delimiter=',').to_numpy().flatten()

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train_label_MBTI = le.fit_transform(train_label_MBTI)
test_label_MBTI = le.transform(test_label_MBTI)

inception_test_label = le.transform(inception_test_label)
print(le.classes_)

['ENFJ' 'ENFP' 'ENTJ' 'ENTP' 'ESFJ' 'ESFP' 'ESTJ' 'ESTP' 'INFJ' 'INFP'
 'INTJ' 'INTP' 'ISFJ' 'ISFP' 'ISTJ' 'ISTP']


In [4]:
def preprocess_image(path):
    img = tf.keras.preprocessing.image.load_img(path, target_size=(300, 300))
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = tf.keras.applications.inception_resnet_v2.preprocess_input(x)
    return x

In [5]:
import numpy as np

df_train_MBTI = pd.read_csv("csv\\original\\train_MBTI.csv", usecols=[0], delimiter=',').values.flatten()
df_test_MBTI = pd.read_csv("csv\\original\\test_MBTI.csv", usecols=[0], delimiter=',').values.flatten()

df_inception_test = pd.read_csv("csv\\test\\MBTI_label.csv", usecols=[0], delimiter=',').values.flatten()

preprocessed_train_images_MBTI = np.vstack([preprocess_image(df_train_MBTI[i]) for i in range(df_train_MBTI.shape[0])])
preprocessed_test_images_MBTI = np.vstack([preprocess_image(df_test_MBTI[i]) for i in range(df_test_MBTI.shape[0])])

preprocessed_inception_test_images = np.vstack([preprocess_image(df_inception_test[i]) for i in range(df_inception_test.shape[0])])

In [7]:
print(sys.executable)

train_data_MBTI = model.predict(preprocessed_train_images_MBTI)
train_data_MBTI = train_data_MBTI.reshape(train_label_MBTI.shape[0], -1)

test_data_MBTI = model.predict(preprocessed_test_images_MBTI)
test_data_MBTI = test_data_MBTI.reshape(test_label_MBTI.shape[0], -1)

inception_test_data = model.predict(preprocessed_inception_test_images)
inception_test_data = inception_test_data.reshape(inception_test_data.shape[0], -1)

C:\Users\leeh9\anaconda3\envs\ugrp_3\python.exe


In [8]:
np.save("inception\\original\\train_data_MBTI.npy", train_data_MBTI)
np.save("inception\\original\\test_data_MBTI.npy", test_data_MBTI)
np.save("inception\\original\\train_label_MBTI.npy", train_label_MBTI)
np.save("inception\\original\\test_label_MBTI.npy", test_label_MBTI)

np.save("inception\\test\\test_data_MBTI.npy", inception_test_data)
np.save("inception\\test\\test_label_MBTI.npy", inception_test_label)